# Format raw and processed files for GEO uploads

Note: This is written to be run on a Mac with the research drive mounted. Adjust file paths as necessary if this is not the case.

## Setup

In [1]:
from collections import namedtuple
import pandas as pd
import os
import hashlib

# Setting up this named tuple to store any further information we need about the runs
# (such as directory paths, etc.)
RunInfo = namedtuple("RunInfo", ["name", "fastq_folder"])

runs = [
    RunInfo(
        "Run230316",
        "/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_VL00320_43_AACJYMYM5/Analysis/1/Data/fastq",
    ),
    RunInfo(
        "Run230322",
        "/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230322_VL00320_45_AACJYNCM5/Analysis/1/Data/fastq",
    ),
    RunInfo(
        "Run230323",
        "/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_run/fastq",
    ),
]

experiment_tag = "230324-3way-merge"

# Path to processed data files
processed_data_folder = f"/Volumes/kueck/Pipeline/CellLines/RNAseq/{experiment_tag}/star_salmon"
processed_data_file_names = [
    "salmon.merged.gene_tpm.tsv",
    "salmon.merged.gene_counts.tsv",
]

# Path to fastq folder where all the fastq files will be copied to
# Note that this copies straight into the research drive so that the fastq files are not on local device
final_fastq_folder = f"/Volumes/kueck/ovarian_cancer_cisplatin_response_manuscript/geo_metadata/data/{experiment_tag}/fastq"

# Paths to save the sample/processed data info
sample_info_path = f"../data/{experiment_tag}/raw_sample_info.csv"
processed_data_info_path = f"../data/{experiment_tag}/processed_data_info.csv"
formatted_sample_files_path = f"../data/{experiment_tag}/raw_sample_files.csv"

# Print command and run command
def run_command(cmd):
    print(f"cmd: {cmd}")
    os.system(cmd)

# DataFrame to hold info about each sample
sample_info = pd.DataFrame(
    columns=["sample_id", "original_fastq_path", "symlink_fastq_path", "final_file_name", "md5_checksum"]
)
sample_info.set_index("sample_id", inplace=True)

In [2]:
def compute_md5_checksum(file_path):
    md5_hash = hashlib.md5()
    with open(file_path, "rb") as file:
        # Read the file in chunks to avoid memory issues with large files
        for chunk in iter(lambda: file.read(4096), b""):
            md5_hash.update(chunk)
    return md5_hash.hexdigest()

## Collect information about each sample

In [3]:
for run in runs:
    for sample_num in range(1, 34):
        for read_num in range(1, 3):
            # Infer the original fastq path
            original_fastq_path = f"{run.fastq_folder}/{sample_num}_S{sample_num}_R{read_num}_001.fastq.gz"

            # Create a unique sample_id including the run tag
            sample_id = f"{sample_num}_S{sample_num}_R{read_num}_001_{run.name}"

            # Add sample info to data frame
            sample_info.loc[sample_id] = [original_fastq_path, "", "", ""]

display(sample_info)

,original_fastq_path,symlink_fastq_path,final_file_name,md5_checksum
sample_id,,,,
1_S1_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,
1_S1_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,
2_S2_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,
2_S2_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,
3_S3_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,
...,...,...,...,...
31_S31_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,,,
32_S32_R1_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,,,
32_S32_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,,,


## Compute MD5 checksums

In [4]:
i = 1
for sample_id, row in sample_info.iterrows():
    display(f"Computing MD5 for {sample_id} ({i}/{len(sample_info)})")
    sample_original_path = row["original_fastq_path"]

    # Compute md5 checksum
    md5_checksum = compute_md5_checksum(sample_original_path)

    # Update sample info
    sample_info.at[sample_id, "md5_checksum"] = md5_checksum

    i += 1

display(sample_info)

'Computing MD5 for 1_S1_R1_001_Run230316 (1/198)'

'Computing MD5 for 1_S1_R2_001_Run230316 (2/198)'

'Computing MD5 for 2_S2_R1_001_Run230316 (3/198)'

'Computing MD5 for 2_S2_R2_001_Run230316 (4/198)'

'Computing MD5 for 3_S3_R1_001_Run230316 (5/198)'

...

'Computing MD5 for 32_S32_R1_001_Run230323 (195/198)'

'Computing MD5 for 32_S32_R2_001_Run230323 (196/198)'

'Computing MD5 for 33_S33_R1_001_Run230323 (197/198)'

'Computing MD5 for 33_S33_R2_001_Run230323 (198/198)'

,original_fastq_path,symlink_fastq_path,final_file_name,md5_checksum
sample_id,,,,
1_S1_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,19cee84a6a3598930e2e6befdf203e0f
1_S1_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,a41a73120d75d1a5c4696488dc56af78
2_S2_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,0613d63ebd5346ef57124d458a5e9f7b
2_S2_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,46e40cbfdf13d025863609aab40e63cd
3_S3_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,,,e28b40d675a63b1837587ff28dc2a618
...,...,...,...,...
31_S31_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,,,267e934f226bb6bc9926fdcd69a8b063
32_S32_R1_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,,,c8634cdab2082b4198d58ccc0057f77b
32_S32_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,,,b332e910a17b49a3f1599ee04b4d2743


## Make a symlink to fastq files

In [8]:
i = 1
for sample_id, row in sample_info.iterrows():
    display(f"Making symlink for fastq file {sample_id} ({i}/{len(sample_info)})")

    # Create symbolic link
    sample_original_path = row["original_fastq_path"]
    final_file_name = f"{sample_id}.fastq.gz"
    symlink_fastq_path = f"{final_fastq_folder}/{sample_id}.fastq.gz"
    os.symlink(sample_original_path, symlink_fastq_path)

    # Update sample info
    sample_info.at[sample_id, "symlink_fastq_path"] = symlink_fastq_path
    sample_info.at[sample_id, "final_file_name"] = final_file_name

    i += 1

display(sample_info)

'Making symlink for fastq file 1_S1_R1_001_Run230316 (1/198)'

'Making symlink for fastq file 1_S1_R2_001_Run230316 (2/198)'

'Making symlink for fastq file 2_S2_R1_001_Run230316 (3/198)'

'Making symlink for fastq file 2_S2_R2_001_Run230316 (4/198)'

'Making symlink for fastq file 3_S3_R1_001_Run230316 (5/198)'

...

'Making symlink for fastq file 32_S32_R1_001_Run230323 (195/198)'

'Making symlink for fastq file 32_S32_R2_001_Run230323 (196/198)'

'Making symlink for fastq file 33_S33_R1_001_Run230323 (197/198)'

'Making symlink for fastq file 33_S33_R2_001_Run230323 (198/198)'

,original_fastq_path,symlink_fastq_path,final_file_name,md5_checksum
sample_id,,,,
1_S1_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,1_S1_R1_001_Run230316.fastq.gz,19cee84a6a3598930e2e6befdf203e0f
1_S1_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,1_S1_R2_001_Run230316.fastq.gz,a41a73120d75d1a5c4696488dc56af78
2_S2_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,2_S2_R1_001_Run230316.fastq.gz,0613d63ebd5346ef57124d458a5e9f7b
2_S2_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,2_S2_R2_001_Run230316.fastq.gz,46e40cbfdf13d025863609aab40e63cd
3_S3_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,3_S3_R1_001_Run230316.fastq.gz,e28b40d675a63b1837587ff28dc2a618
...,...,...,...,...
31_S31_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,31_S31_R2_001_Run230323.fastq.gz,267e934f226bb6bc9926fdcd69a8b063
32_S32_R1_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,32_S32_R1_001_Run230323.fastq.gz,c8634cdab2082b4198d58ccc0057f77b
32_S32_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,32_S32_R2_001_Run230323.fastq.gz,b332e910a17b49a3f1599ee04b4d2743


In [ ]:
# Code for deep copy:

# i = 1
# for sample_id, row in sample_info.iterrows():
#     display(f"Copying fastq file {sample_id} ({i}/{len(sample_info)})")
#     sample_original_path = row["original_fastq_path"]
#     copied_fastq_path = f"{final_fastq_folder}/{sample_id}.fastq.gz"

#     # Copy the fastq file
#     run_command(f"cp {sample_original_path} {copied_fastq_path}")

#     # Compute md5 checksum on copied file
#     display(f"Computing MD5 for file {sample_id} ({i}/{len(sample_info)})")
#     copied_md5_hash = compute_md5_checksum(copied_fastq_path)

#     # Check md5 checksum matches
#     assert (copied_md5_checksum == row["md5_checksum"], f"MD5 checksums do not match for {sample_id}")

#     # Update sample info
#     sample_info.at[sample_id, "copied_fastq_path"] = copied_fastq_path

#     i += 1
#     break

# display(sample_info)

## Save sample info

This can then be copied into the GEO metadata spreadsheet

In [10]:
sample_info.to_csv(sample_info_path, sep="\t", index=True)
display(sample_info)

,original_fastq_path,symlink_fastq_path,final_file_name,md5_checksum
sample_id,,,,
1_S1_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,1_S1_R1_001_Run230316.fastq.gz,19cee84a6a3598930e2e6befdf203e0f
1_S1_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,1_S1_R2_001_Run230316.fastq.gz,a41a73120d75d1a5c4696488dc56af78
2_S2_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,2_S2_R1_001_Run230316.fastq.gz,0613d63ebd5346ef57124d458a5e9f7b
2_S2_R2_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,2_S2_R2_001_Run230316.fastq.gz,46e40cbfdf13d025863609aab40e63cd
3_S3_R1_001_Run230316,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230316_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,3_S3_R1_001_Run230316.fastq.gz,e28b40d675a63b1837587ff28dc2a618
...,...,...,...,...
31_S31_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,31_S31_R2_001_Run230323.fastq.gz,267e934f226bb6bc9926fdcd69a8b063
32_S32_R1_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,32_S32_R1_001_Run230323.fastq.gz,c8634cdab2082b4198d58ccc0057f77b
32_S32_R2_001_Run230323,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/230323_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,32_S32_R2_001_Run230323.fastq.gz,b332e910a17b49a3f1599ee04b4d2743


## Create formatted table with files per experimental sample

In [17]:
columns = ["sample_number"]
for run in runs:
    for read_num in range(1, 3):
        columns.append(f"{run.name}_R{read_num}")

formatted_sample_info = pd.DataFrame(
    columns=columns
)
formatted_sample_info.set_index("sample_number", inplace=True)

for sample_num in range(1, 34):
    for run in runs:
        for read_num in range(1, 3):
            column_name = f"{run.name}_R{read_num}"
            file_name = f"{sample_num}_S{sample_num}_R{read_num}_001_{run.name}.fastq.gz"

            # Check that this file name appears in the sample_info data frame
            assert(file_name in sample_info["final_file_name"].values)

            formatted_sample_info.at[sample_num, column_name] = file_name

display(formatted_sample_info)

formatted_sample_info.to_csv(formatted_sample_files_path, sep="\t", index=True)

,Run230316_R1,Run230316_R2,Run230322_R1,Run230322_R2,Run230323_R1,Run230323_R2
sample_number,,,,,,
1,1_S1_R1_001_Run230316.fastq.gz,1_S1_R2_001_Run230316.fastq.gz,1_S1_R1_001_Run230322.fastq.gz,1_S1_R2_001_Run230322.fastq.gz,1_S1_R1_001_Run230323.fastq.gz,1_S1_R2_001_Run230323.fastq.gz
2,2_S2_R1_001_Run230316.fastq.gz,2_S2_R2_001_Run230316.fastq.gz,2_S2_R1_001_Run230322.fastq.gz,2_S2_R2_001_Run230322.fastq.gz,2_S2_R1_001_Run230323.fastq.gz,2_S2_R2_001_Run230323.fastq.gz
3,3_S3_R1_001_Run230316.fastq.gz,3_S3_R2_001_Run230316.fastq.gz,3_S3_R1_001_Run230322.fastq.gz,3_S3_R2_001_Run230322.fastq.gz,3_S3_R1_001_Run230323.fastq.gz,3_S3_R2_001_Run230323.fastq.gz
4,4_S4_R1_001_Run230316.fastq.gz,4_S4_R2_001_Run230316.fastq.gz,4_S4_R1_001_Run230322.fastq.gz,4_S4_R2_001_Run230322.fastq.gz,4_S4_R1_001_Run230323.fastq.gz,4_S4_R2_001_Run230323.fastq.gz
5,5_S5_R1_001_Run230316.fastq.gz,5_S5_R2_001_Run230316.fastq.gz,5_S5_R1_001_Run230322.fastq.gz,5_S5_R2_001_Run230322.fastq.gz,5_S5_R1_001_Run230323.fastq.gz,5_S5_R2_001_Run230323.fastq.gz
6,6_S6_R1_001_Run230316.fastq.gz,6_S6_R2_001_Run230316.fastq.gz,6_S6_R1_001_Run230322.fastq.gz,6_S6_R2_001_Run230322.fastq.gz,6_S6_R1_001_Run230323.fastq.gz,6_S6_R2_001_Run230323.fastq.gz
7,7_S7_R1_001_Run230316.fastq.gz,7_S7_R2_001_Run230316.fastq.gz,7_S7_R1_001_Run230322.fastq.gz,7_S7_R2_001_Run230322.fastq.gz,7_S7_R1_001_Run230323.fastq.gz,7_S7_R2_001_Run230323.fastq.gz
8,8_S8_R1_001_Run230316.fastq.gz,8_S8_R2_001_Run230316.fastq.gz,8_S8_R1_001_Run230322.fastq.gz,8_S8_R2_001_Run230322.fastq.gz,8_S8_R1_001_Run230323.fastq.gz,8_S8_R2_001_Run230323.fastq.gz
9,9_S9_R1_001_Run230316.fastq.gz,9_S9_R2_001_Run230316.fastq.gz,9_S9_R1_001_Run230322.fastq.gz,9_S9_R2_001_Run230322.fastq.gz,9_S9_R1_001_Run230323.fastq.gz,9_S9_R2_001_Run230323.fastq.gz


## Compute MD5 Checksums for processed data files

In [19]:
processed_file_info = pd.DataFrame(
    columns=["file_name", "md5_checksum"]
)

for file_name in processed_data_file_names:
    file_path = f"{processed_data_folder}/{file_name}"

    # Compute md5 checksum
    md5_checksum = compute_md5_checksum(file_path)

    # Add to processed file info
    processed_file_info.loc[file_name] = [file_name, md5_checksum]

display(processed_file_info)

# Save processed data info
processed_file_info.to_csv(processed_data_info_path, sep="\t", index=False)

,file_name,md5_checksum
salmon.merged.gene_tpm.tsv,salmon.merged.gene_tpm.tsv,69df151d0b83454cc8c36b6972e62629
salmon.merged.gene_counts.tsv,salmon.merged.gene_counts.tsv,8955cd3fcad46e7cb9b11e12b97bd9e8
